In [2]:
!pip install transformers
!pip install Flask  


In [11]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
pip install datasets

   ---------------------------------------- 0.0/507.1 kB ? eta -:--:--
   -- ------------------------------------- 30.7/507.1 kB 1.4 MB/s eta 0:00:01
   --- ----------------------------------- 41.0/507.1 kB 653.6 kB/s eta 0:00:01
   --- ----------------------------------- 41.0/507.1 kB 653.6 kB/s eta 0:00:01
   ---- ---------------------------------- 61.4/507.1 kB 328.2 kB/s eta 0:00:02
   ----- --------------------------------- 71.7/507.1 kB 328.6 kB/s eta 0:00:02
   -------- ----------------------------- 112.6/507.1 kB 437.6 kB/s eta 0:00:01
   -------- ----------------------------- 112.6/507.1 kB 437.6 kB/s eta 0:00:01
   --------- ---------------------------- 122.9/507.1 kB 361.0 kB/s eta 0:00:02
   --------- ---------------------------- 122.9/507.1 kB 361.0 kB/s eta 0:00:02
   ------------- ------------------------ 174.1/507.1 kB 403.5 kB/s eta 0:00:01
   ----------------- -------------------- 235.5/507.1 kB 497.3 kB/s eta 0:00:01
   ----------------- -------------------- 235.5/50

In [5]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")  # Load CNN/DailyMail dataset


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

# Load your dataset (e.g., CNN/DailyMail)
dataset = load_dataset("cnn_dailymail", "3.0.0")
train_texts = dataset['train']['article'][:1000]  # Adjust the dataset size as needed

# Load pre-trained model and tokenizer
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset
tokenized_dataset = tokenizer(train_texts, truncation=True, padding=True)

# Define a custom collate function
def collate_fn(batch):
    return tokenizer(batch['input_ids'], truncation=True, padding=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./custom_summarizer',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
)

# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=collate_fn,
)

#trainer.train()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
from flask import Flask, render_template, request
from transformers import T5ForConditionalGeneration, T5Tokenizer

app = Flask(__name__)

# Load your trained summarization model and tokenizer
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define personalized prompt templates
prompt_templates = {
    'objective': 'In an objective manner, summarize the article: {}',
    'factual': 'Provide a factual summary of the article: {}',
    'humorous': 'Summarize the article in a humorous way: {}',
}

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/summarize', methods=['POST'])
def summarize():
    article = request.form['article']
    summary_length = int(request.form['summary_length'])
    style = request.form['style']

    # Design personalized prompt based on user choices
    prompt = prompt_templates.get(style, 'Summarize the article: {}').format(article)

    # Generate personalized summary
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(**inputs, max_length=summary_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    personalized_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return render_template('index.html', article=article, personalized_summary=personalized_summary)

if __name__ == '__main__':
    app.run(debug=True)


C:\Users\athit\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\t5\tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing change

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\athit\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
pip install accelerate -U



   ---------------------------------------- 0.0/270.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/270.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/270.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/270.9 kB 325.1 kB/s eta 0:00:01
   ----- --------------------------------- 41.0/270.9 kB 217.9 kB/s eta 0:00:02
   -------- ------------------------------ 61.4/270.9 kB 297.7 kB/s eta 0:00:01
   ---------- ---------------------------- 71.7/270.9 kB 280.5 kB/s eta 0:00:01
   ------------- ------------------------- 92.2/270.9 kB 348.6 kB/s eta 0:00:01
   --------------- ---------------------- 112.6/270.9 kB 327.2 kB/s eta 0:00:01
   --------------- ---------------------- 112.6/270.9 kB 327.2 kB/s eta 0:00:01
   -------------------- ----------------- 143.4/270.9 kB 340.5 kB/s eta 0:00:01
   --------------------- ---------------- 153.6/270.9 kB 352.2 kB/s eta 0:00:01
   ------------------------ ------------- 174.1/270.9 kB 349.3 k